In [ ]:
import pandas as pd
#Loading the dataset into a dataframe
df = pd.read_csv('C:/Users/sheth/Desktop/DSMP/simulated_transaction_2024.csv')

#Having a look at the data
print(df.head())


In [ ]:
#Frequency of transactions for each unique value in 'Third party name' column
transaction_counts = df['Third Party Name'].value_counts()
pd.set_option('display.max_rows', None)
print(transaction_counts)

Feature Engineering as done in Dataset 1

In [ ]:
#Defining a function to create a new feature to catagorize the transactions into filtered categories
def categorize(name):
    if name in ['Sports Direct', 'Topshop', 'Fat Face', 'Matalan', 'AMAZON', 'Blackwell\'s','Reebok', 'JD Sports', 'North Face', 'Boots', 'Lloyds Pharmacy']:
        return 'Retail Stores'
    elif name in ['Netflix', 'Disney', 'Blizzard', 'Mojang Studios', 'Amazon', 'Xbox', 'Gamestation']:
        return 'Entertainment & Media'
    elif name in ['Sainsbury', 'Tesco', 'Coop Local', 'Sainsbury Local']:
        return 'Grocery'
    elif name in ['JustEat', 'Deliveroo', 'Starbucks', 'Five Senses Art', 'Coffee #1', 'Costa Coffee', 'Jollyes']:
        return 'Food & Dining'
    elif name in ['Halifax', 'LBG', 'Premier Finance', 'CPA']:
        return 'Financial Services'
    elif name in ['PureGym', 'Grand Union BJJ', 'Selfridges', 'Lloyds Pharmacy', 'Vision Express', 'Pets Corner']:
        return 'Health & Wellness'
    elif name in ['Blackwell\'s', 'Brilliant Brushes','Craftastic', 'A Yarn Story', 'Cass Art', 'Foyles']:
        return 'Education & Books'
    elif name in ['The Works', 'Loosely Fitted', 'Wool', 'Hobby Lobby', 'Hobbycraft', 'Happy Days Home', 'Lavender Fields']:
        return 'Home & Lifestyle'
    else:
        return 'Other Services'

#Creating a new feature 'Category' based on the function
df['Category'] = df['Third Party Name'].apply(categorize)

#DataFrame with the new 'Category' column
print(df)


In [ ]:
#To check which Third Parties havent been categorised 
#Transactions where 'Category' is 'Other Services' and display all values
other_services_df = df[df['Category'] == 'Other Services']
print(other_services_df['Third Party Name'].unique())

In [ ]:
#Convert 'Amount' column to numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

#Average, minimum, and maximum spends
category_stats = df.groupby('Category')['Amount'].agg(['mean', 'min', 'max'])

# Display the statistics
print(category_stats)

#Function for the transaction with minimum spend
def get_min_transaction_details(group):
    min_transaction = group.loc[group['Amount'] == group['Amount'].min()]
    return min_transaction[['Third Party Name', 'Amount']]

#Function for the transaction with maximum spend
def get_max_transaction_details(group):
    max_transaction = group.loc[group['Amount'] == group['Amount'].max()]
    return max_transaction[['Third Party Name', 'Amount']]

#Transactions with min and max spends for each category
min_transaction_details = df.groupby('Category').apply(get_min_transaction_details)
max_transaction_details = df.groupby('Category').apply(get_max_transaction_details)

#Statistics and transaction details
for category, stats in category_stats.iterrows():
    print(f"Category: {category}")
    print(f"Average Spend: {stats['mean']:.2f}")
    print(f"Minimum Spend: {stats['min']:.2f}")
    print("Details of Minimum Spend Transaction:")
    print(min_transaction_details.loc[category])
    print(f"Maximum Spend: {stats['max']:.2f}")
    print("Details of Maximum Spend Transaction:")
    print(max_transaction_details.loc[category])
    print("\n")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#Average, minimum, and maximum spends
category_stats = df.groupby('Category')['Amount'].agg(['mean', 'min', 'max']).round(2)

#Plotting mean of transactions by Category
plt.figure(figsize=(10, 6))
plt.bar(category_stats.index, category_stats['mean'], color='skyblue')
plt.title('Mean Transaction Amounts by Category')
plt.xlabel('Category')
plt.ylabel('Mean Amount')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#Plotting min of transactions by Category
plt.figure(figsize=(10, 6))
plt.bar(category_stats.index, category_stats['min'], color='lightgreen')
plt.title('Minimum Transaction Amounts by Category')
plt.xlabel('Category')
plt.ylabel('Minimum Amount')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#Plotting max of transactions by Category
plt.figure(figsize=(10, 6))
plt.bar(category_stats.index, category_stats['max'], color='salmon')
plt.title('Maximum Transaction Amounts by Category')
plt.xlabel('Category')
plt.ylabel('Maximum Amount')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
#Variance for each category
category_variance = df.groupby('Category')['Amount'].var()

#Rounding the variances to three decimal places
category_variance = category_variance.round(3)

#Variance for each category
print(category_variance)

In [ ]:
#Plotting variances by Categoeies
category_variance.plot(kind='bar', color='skyblue', figsize=(10, 6))
plt.title('Variance of Transaction Amounts by Category')
plt.xlabel('Category')
plt.ylabel('Variance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### Proceeding with RFM Analysis.
 This analysis involves allocationn of individual scores based on:
 * (R) recency
 * (F) frequency
 * (M) Monetary  
 
 We have used the metric, 'Quantile' to calculate the score of each of the 3 aspects, where lower than 25% is given a score of 4, lower than 50%, a 3 and so on. After calculating all the individual scores, we sum them to get a RFM score, the customers with the best RFM scores are our target customers for any new services, projects, etc

In [ ]:
df.head()

In [ ]:
#Converting 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

#Calculating recency, frequency, and monetary value for each transaction
snapshot_date = df['Date'].max() + pd.DateOffset(1)  
rfm_df = df.groupby('Account No').agg({
    'Date': lambda x: (snapshot_date - x.max()).days,  
    'Account No': 'count',                             
    'Amount': 'sum'                                    
}).rename(columns={'Date': 'Recency', 'Account No': 'Frequency', 'Amount': 'Monetary'})

#Calculating RFM scores for each transaction
rfm_scores = df.groupby('Account No').agg({
    'Date': lambda x: (snapshot_date - x.max()).days,  
    'Account No': 'count',                             
    'Amount': 'sum'                                    
}).rename(columns={'Date': 'Recency', 'Account No': 'Frequency', 'Amount': 'Monetary'})

#Defining quantiles for RFM scores
quantiles = rfm_scores.quantile(q=[0.25, 0.5, 0.75])

#Assigning scores based on quantiles
def assign_rfm_score(x, c, quantiles):
    if x <= quantiles[c][0.25]:
        return 4
    elif x <= quantiles[c][0.50]:
        return 3
    elif x <= quantiles[c][0.75]: 
        return 2
    else:
        return 1

rfm_scores['R_Score'] = rfm_scores['Recency'].apply(assign_rfm_score, args=('Recency', quantiles))
rfm_scores['F_Score'] = rfm_scores['Frequency'].apply(assign_rfm_score, args=('Frequency', quantiles))
rfm_scores['M_Score'] = rfm_scores['Monetary'].apply(assign_rfm_score, args=('Monetary', quantiles))

#RFM scores for each transaction
print(rfm_scores)


Following are the Recency, Frequency and Monetary Scores for the unique accounts of the dataset

In [ ]:
#RFM scores
print(rfm_scores[['R_Score', 'F_Score', 'M_Score']])

Summing up RFM Scores to get best customers from the dataset

In [ ]:

#Calculate Total RFM score
rfm_scores['RFM_Score'] = rfm_scores['R_Score'] + rfm_scores['F_Score'] + rfm_scores['M_Score']

#Sorting customers by RFM score in descending order to get the best customers
best_customers = rfm_scores.sort_values(by='RFM_Score', ascending=False)

#Best customers
print(best_customers.head())


In [ ]:
#Top 1000 customers
top_1000_customers = best_customers.head(1000)
print(top_1000_customers)

### Understanding the distribution of Actual number of accounts over the transactions, to identify the High-Value, Loyal and Lost customers.
* High-Value Customers: Account holders with high Frequency and Monetary Transactions, low Recency
* Loyal Customers: Account holders with high Recency and Frequency Transactions, low Monetary
* Lost Customers: Account holders with low Recency and Frequency Transactions, high Monetary 

In [ ]:
df.shape

In [ ]:
unique_accounts_count = len(df['Account No'].value_counts())
print("Unique number of accounts:", unique_accounts_count)


In [ ]:
df.head()

In [ ]:
#Threshold values
recency_threshold_high = 2  
frequency_threshold_high = 2  
monetary_threshold_high = 2  

frequency_threshold_loyal = 2  
monetary_threshold_loyal = 2  

recency_threshold_lost = 3  
frequency_threshold_lost = 3  
monetary_threshold_lost = 3  

#Reseting the 'RFM_Segment' column
rfm_scores['RFM_Segment'] = ''

# Assign segment labels based on the adjusted thresholds to the entire dataset
rfm_scores.loc[(rfm_scores['R_Score'] <= recency_threshold_high) & 
               (rfm_scores['F_Score'] >= frequency_threshold_high) & 
               (rfm_scores['M_Score'] >= monetary_threshold_high), 'RFM_Segment'] = 'High-Value Customer'

rfm_scores.loc[(rfm_scores['F_Score'] >= frequency_threshold_loyal) & 
               (rfm_scores['R_Score'] >= recency_threshold_high) & 
               (rfm_scores['M_Score'] <= monetary_threshold_lost) &
               (rfm_scores['RFM_Segment'] == ''), 'RFM_Segment'] = 'Loyal Customer'

rfm_scores.loc[(rfm_scores['R_Score'] <= recency_threshold_lost) & 
               (rfm_scores['F_Score'] <= frequency_threshold_lost) & 
               (rfm_scores['M_Score'] >= monetary_threshold_lost) &
               (rfm_scores['RFM_Segment'] == ''), 'RFM_Segment'] = 'Lost Customer'

#Number of customers in each segment
segment_counts = rfm_scores['RFM_Segment'].value_counts()
print(segment_counts)


Clustering: K-means clustering and DBSCAN can be applied to segment customers based on their transactional behavior.By clustering customers based on transaction attributes such as Balance, Amount, and Category, we can identify groups of customers with similar spending patterns or preferences. Eg: High spenders, frequent shoppers, or users of specific services.

In [ ]:
df.shape

In [ ]:
missing_values = df.isnull().sum()
print(missing_values)


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

#Dropping rows with missing values in Date, Timestamp, and Account No columns
df.dropna(subset=["Date", "Timestamp", "Account No"], inplace=True)

#Imputing missing values in numeric columns (Amount, Balance) with median
numeric_cols = ["Amount", "Balance"]
numeric_imputer = SimpleImputer(strategy="median")
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

In [ ]:
#Data types of all columns
print(df.dtypes)


In [ ]:
#Converting "Timestamp" column to datetime64[ns]
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
print(df.dtypes)

In [ ]:
import pandas as pd

#Extracting features from Date and Timestamp columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute

#Dropping the original Date and Timestamp columns
df.drop(columns=['Date', 'Timestamp'], inplace=True)

#Applying one-hot encoding to the Category column
df_encoded = pd.get_dummies(df, columns=['Category'], drop_first=True)

#Dataframe with encoded columns
print(df_encoded)


In [ ]:
#Dropping rows with NaN values
df_cleaned = df_without_name.dropna()

#Standardization
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df_cleaned)

#Initializing KMeans model
kmeans = KMeans(n_clusters=k, random_state=42)

#Fitting KMeans model to the data
kmeans.fit(scaled_df)

#Cluster labels
cluster_labels = kmeans.labels_

#Concatinating cluster labels to the dataframe
df_cleaned['Cluster'] = cluster_labels

#Dataframe with cluster labels
print(df_cleaned)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Visualizing Balance and Amount features with clusters
feature1 = 'Balance'
feature2 = 'Amount'

#Plotting the data points
plt.figure(figsize=(8, 6))
sns.scatterplot(x=feature1, y=feature2, hue='Cluster', data=df_cleaned, palette='viridis', legend='full')
plt.title('Clustering Output')
plt.xlabel(feature1)
plt.ylabel(feature2)

#Adding cluster centers
if hasattr(kmeans, 'cluster_centers_'):
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)  
    plt.scatter(cluster_centers[:, df_cleaned.columns.get_loc(feature1)], 
                cluster_centers[:, df_cleaned.columns.get_loc(feature2)], 
                marker='x', s=100, c='black', label='Cluster Centers')

plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

#Initializing DBSCAN model
dbscan = DBSCAN(eps=0.5, min_samples=5)

#Fitting DBSCAN model to the data
dbscan.fit(scaled_df)

#Cluster labels
cluster_labels = dbscan.labels_

#Cluster labels to the dataframe
df_cleaned['Cluster'] = cluster_labels

#Dataframe with cluster labels
print(df_cleaned)

#Visualizing the clusters
plt.scatter(df_cleaned['Balance'], df_cleaned['Amount'], c=cluster_labels, cmap='viridis')
plt.xlabel('Balance')
plt.ylabel('Amount')
plt.title('DBSCAN Clustering')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

#Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

#Plot for Balance and Amount
axs[0].scatter(df_cleaned['Balance'], df_cleaned['Amount'], c=cluster_labels, cmap='viridis')
axs[0].set_xlabel('Balance')
axs[0].set_ylabel('Amount')
axs[0].set_title('Balance vs Amount')

#Plot for Account No and Amount
axs[1].scatter(df_cleaned['Account No'], df_cleaned['Amount'], c=cluster_labels, cmap='viridis')
axs[1].set_xlabel('Account No')
axs[1].set_ylabel('Amount')
axs[1].set_title('Account No vs Amount')

#Plot for Account No and Balance
axs[2].scatter(df_cleaned['Account No'], df_cleaned['Balance'], c=cluster_labels, cmap='viridis')
axs[2].set_xlabel('Account No')
axs[2].set_ylabel('Balance')
axs[2].set_title('Account No vs Balance')

#Visualizing the Plots
plt.tight_layout()
plt.show()


In [ ]:
plt.scatter(df_cleaned['Balance'], df_cleaned['Amount'], c=cluster_labels, cmap='viridis')
plt.xlabel('Balance')
plt.ylabel('Amount')
plt.title('Balance vs Amount (Clustered)')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
plt.hist(df_cleaned['Amount'], bins=30, color='skyblue', alpha=0.7)
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.title('Transaction Amount Distribution (Clustered)')
plt.show()


Visualizing the optimum number of clusters using the Elbow method

In [ ]:
from sklearn.cluster import KMeans

#To define the optimum number of clusters 
k_values = range(1, 11)
wcss = []

#Calculating WCSS for each k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_df)
    wcss.append(kmeans.inertia_)

#Plotting the WCSS against the number of clusters
plt.plot(k_values, wcss, marker='*')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method')
plt.xticks(k_values)
plt.show()


In [ ]:
#Moving with 3 clusters
k = 3

#Reinitializing and refitting KMeans model with 3 clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(scaled_df)

#Updating cluster labels
cluster_labels = kmeans.labels_

#Updating dataframe with cluster labels
df_cleaned['Cluster'] = cluster_labels

#Dataframe with updated cluster labels
print(df_cleaned)

#Visualizing Balance and Amount features with 3 clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=feature1, y=feature2, hue='Cluster', data=df_cleaned, palette='viridis', legend='full')
plt.title('Clustering Output (3 Clusters)')
plt.xlabel(feature1)
plt.ylabel(feature2)

#Cluster centers
if hasattr(kmeans, 'cluster_centers_'):
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)  
    plt.scatter(cluster_centers[:, df_cleaned.columns.get_loc(feature1)], 
                cluster_centers[:, df_cleaned.columns.get_loc(feature2)], 
                marker='x', s=100, c='black', label='Cluster Centers')

plt.legend()
plt.show()


In [ ]:
#Moving with 4 clusters
k = 4

#Reinitializing and refitting KMeans model with 4 clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(scaled_df)

#Updating cluster labels
cluster_labels = kmeans.labels_

#Updating dataframe with cluster labels
df_cleaned['Cluster'] = cluster_labels

#Dataframe with updated cluster labels
print(df_cleaned)

#Visualizing Balance and Amount features with 4 clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=feature1, y=feature2, hue='Cluster', data=df_cleaned, palette='viridis', legend='full')
plt.title('Clustering Output (4 Clusters)')
plt.xlabel(feature1)
plt.ylabel(feature2)

#Cluster centers
if hasattr(kmeans, 'cluster_centers_'):
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)  
    plt.scatter(cluster_centers[:, df_cleaned.columns.get_loc(feature1)], 
                cluster_centers[:, df_cleaned.columns.get_loc(feature2)], 
                marker='x', s=100, c='black', label='Cluster Centers')

plt.legend()
plt.show()


In [ ]:
#Moving with 5 clusters
k = 5

#Reinitialize and refit KMeans model with 5 clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(scaled_df)

#Updating cluster labels
cluster_labels = kmeans.labels_

#Updating dataframe with cluster labels
df_cleaned['Cluster'] = cluster_labels

#Dataframe with updated cluster labels
print(df_cleaned)

#Visualizing Balance and Amount features with 5 clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=feature1, y=feature2, hue='Cluster', data=df_cleaned, palette='viridis', legend='full')
plt.title('Clustering Output (5 Clusters)')
plt.xlabel(feature1)
plt.ylabel(feature2)

#Cluster centers
if hasattr(kmeans, 'cluster_centers_'):
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)  
    plt.scatter(cluster_centers[:, df_cleaned.columns.get_loc(feature1)], 
                cluster_centers[:, df_cleaned.columns.get_loc(feature2)], 
                marker='x', s=100, c='black', label='Cluster Centers')

plt.legend()
plt.show()


Time Series Forecasting (ARIMA, LSTM): Time series forecasting techniques such as ARIMA (AutoRegressive Integrated Moving Average) or LSTM (Long Short-Term Memory) neural networks can be used to predict future transaction volumes or identify seasonal trends in customer behavior. They can guide resource allocation, marketing campaign planning, and inventory management.

Customer Churn Prediction: Logistic Regression, Random Forest, or Gradient Boosting Machines (GBM).
These models can forecast the likelihood of customers discontinuing their relationship with the bank based on their transaction history and behavior. By identifying customers at risk of churn, the bank can implement retention strategies to retain valuable customers and reduce churn rates.

Deep Learning for Anomaly Detection: Deep learning models, such as autoencoders or variational autoencoders (VAEs), could be trained on transactional data to detect anomalies or unusual patterns that may indicate fraudulent activities or outliers in customer behavior.